# Inport needed package

- import os, sys # to add the parent directory to the path

In [1]:
import os
import sys
import time

- Using torchvision to create a dataset

In [2]:
# Using torchvision to create a dataset
import cv2
from torchvision import transforms
import torch
from torch.utils.data import random_split, DataLoader
import torchvision

import pandas as pd

- import self library

In [3]:
from train.trainer import ClassifierTrainer as Trainer
import dataset as ds  # type: ignore
import model as md  # type: ignore

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define classification train process

1. Define place where the model is saved

In [4]:
time_str = time.strftime("%Y%m%d_%H%M%S")

2. Define train function

In [5]:
def train_process(dataset, model):
  # define batch_size
  batch_size = 64

  # init train val test ds
  train_val_size = int(0.8 * len(dataset))
  test_size = len(dataset) - train_val_size
  train_ds, test_ds = random_split(dataset, [train_val_size, test_size])

  # define optimizer using Adam and loss function
  optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
  loss_fn = torch.nn.CrossEntropyLoss()

  trainer = Trainer(model, optimizer, loss_fn, random_seed_value=86)
  print('device: ', trainer.device)
  avg_loss, metric = trainer.cross_validate(train_ds, k=5, epochs=10, batch_size=batch_size)
  print('avg_loss: ', avg_loss)

  # score model
  test_dataloader = DataLoader(test_ds, batch_size=batch_size, shuffle=True)
  model_scored = trainer.score(test_dataloader)
  print(f'model_scored: {model_scored:.4f}, avg_accuracy: {100*(1 - model_scored):.4f}')

  # return model scored, train_avg_lost
  return model_scored, avg_loss

3. execute progress

- define the model

In [6]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

- Define tested datasets

In [7]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Train all defined model on each registered datasets

In [8]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for name, dataset in datasets.items():
	for model in models:
		print(f'Running {name} dataset with {model.__class__.__name__}')
		# do the train
		start_time = time.time()
		scored, loss = train_process(dataset, model)
		end_time = time.time()
		total_time = end_time - start_time
		print(f'Finished {name} dataset with {model.__class__.__name__}')
		print('----------------------')

		# save the result
		result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model.__class__.__name__],
			'dataset': [name],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
		})], ignore_index=True)

print('Finished all datasets')

# print the result
print(result_df)


Running gi4e_full dataset with ResNet
device:  cuda
Fold 1/5:
Epoch 1/10, Train Loss: 2.4833, Test Loss: 0.6188, Total Time: 00 hours, 00 minutes, 25 seconds
Epoch 2/10, Train Loss: 0.0142, Test Loss: 0.0655, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 3/10, Train Loss: 0.0028, Test Loss: 0.0067, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 4/10, Train Loss: 0.0005, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 5/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 6/10, Train Loss: 0.0002, Test Loss: 0.0002, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 7/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 18 seconds
Epoch 8/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 9/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 10/10, Train Loss: 0.0001, Test Loss: 0.0001, Total Time: 00 hours, 

C:\Users\nnvuf\AppData\Local\Temp\ipykernel_30568\454607160.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({


Epoch 1/10, Train Loss: 1.9241, Test Loss: 1.4556, Total Time: 00 hours, 00 minutes, 26 seconds
Epoch 2/10, Train Loss: 0.0201, Test Loss: 0.0342, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 3/10, Train Loss: 0.0039, Test Loss: 0.0032, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 4/10, Train Loss: 0.0014, Test Loss: 0.0013, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 5/10, Train Loss: 0.0009, Test Loss: 0.0008, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 6/10, Train Loss: 0.0005, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 7/10, Train Loss: 0.0004, Test Loss: 0.0006, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 8/10, Train Loss: 0.0004, Test Loss: 0.0005, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 9/10, Train Loss: 0.0003, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 19 seconds
Epoch 10/10, Train Loss: 0.0003, Test Loss: 0.0004, Total Time: 00 hours, 00 minutes, 19 seconds
Fold 1/5, Total Test Loss: 0.0004, Fold

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/5, Total Test Loss: 0.1398, Fold accuracy: 86.0247
Fold 2/5:
Epoch 1/10, Train Loss: 0.2538, Test Loss: 0.0246, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 2/10, Train Loss: 0.1247, Test Loss: 0.0086, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 3/10, Train Loss: 0.2869, Test Loss: 0.2947, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 4/10, Train Loss: 0.2238, Test Loss: 0.0117, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 5/10, Train Loss: 0.0630, Test Loss: 0.0464, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 6/10, Train Loss: 0.0875, Test Loss: 0.0791, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 7/10, Train Loss: 0.1197, Test Loss: 0.0603, Total Time: 00 hours, 00 minutes, 20 seconds
Epoch 8/10, Train Loss: 0.0921, Test Loss: 0.1194, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 9/10, Train Loss: 0.0533, Test Loss: 0.0022, Total Time: 00 hours, 00 minutes, 21 seconds
Epoch 10/10, Train Loss: 0.0557, Test Loss: 0.0024, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Fold 1/5, Total Test Loss: 0.1111, Fold accuracy: 88.8943
Fold 2/5:
Epoch 1/10, Train Loss: 0.1555, Test Loss: 0.0067, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 2/10, Train Loss: 0.1023, Test Loss: 0.0149, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 3/10, Train Loss: 0.1250, Test Loss: 0.1439, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 4/10, Train Loss: 0.0917, Test Loss: 0.0623, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 5/10, Train Loss: 0.0981, Test Loss: 0.0411, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 6/10, Train Loss: 0.0696, Test Loss: 0.0432, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 7/10, Train Loss: 0.0810, Test Loss: 0.0635, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 8/10, Train Loss: 0.0951, Test Loss: 0.1195, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 9/10, Train Loss: 0.0832, Test Loss: 0.0707, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 10/10, Train Loss: 0.0682, Test Loss: 0.1457, Total Time: 00 h

c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\nnvuf\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

Fold 3/5, Total Test Loss: 0.2687, Fold accuracy: 98.7992
Fold 4/5:
Epoch 1/10, Train Loss: 0.0916, Test Loss: 0.0745, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 2/10, Train Loss: 0.0773, Test Loss: 0.0128, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 3/10, Train Loss: 0.0719, Test Loss: 0.0140, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 4/10, Train Loss: 0.0785, Test Loss: 0.0291, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 5/10, Train Loss: 0.0565, Test Loss: 0.0070, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 6/10, Train Loss: 0.0487, Test Loss: 0.1059, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 7/10, Train Loss: 0.1470, Test Loss: 0.0699, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 8/10, Train Loss: 0.0981, Test Loss: 0.0055, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 9/10, Train Loss: 0.0741, Test Loss: 0.0125, Total Time: 00 hours, 00 minutes, 16 seconds
Epoch 10/10, Train Loss: 0.0356, Test Loss: 0.0002, Total Time: 00 h

4. print the result

In [9]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000017     99.999863   
1           gi4e_full  DenseNet  0.000099     99.996225   
2           gi4e_full       VGG  0.030500     96.777968   
3       gi4e_raw_eyes    ResNet  0.017712     99.769400   
4       gi4e_raw_eyes  DenseNet  0.007763     99.778149   
5       gi4e_raw_eyes       VGG  0.054540     81.848047   
6  gi4e_detected_eyes    ResNet  0.006875     99.928385   
7  gi4e_detected_eyes  DenseNet  0.000107     99.968702   
8  gi4e_detected_eyes       VGG  0.017090     85.945651   

                 total_time  
0 0 days 00:16:23.117301226  
1 0 days 00:20:05.628188372  
2 0 days 00:19:24.024507523  
3 0 days 00:08:47.432441950  
4 0 days 00:09:55.128549576  
5 0 days 00:15:27.017767429  
6 0 days 00:08:44.347270250  
7 0 days 00:09:53.161640406  
8 0 days 00:15:16.971753836  


In [10]:
print(result_df)

              dataset     model  avg_loss  avg_accuracy  \
0           gi4e_full    ResNet  0.000017     99.999863   
1           gi4e_full  DenseNet  0.000099     99.996225   
2           gi4e_full       VGG  0.030500     96.777968   
3       gi4e_raw_eyes    ResNet  0.017712     99.769400   
4       gi4e_raw_eyes  DenseNet  0.007763     99.778149   
5       gi4e_raw_eyes       VGG  0.054540     81.848047   
6  gi4e_detected_eyes    ResNet  0.006875     99.928385   
7  gi4e_detected_eyes  DenseNet  0.000107     99.968702   
8  gi4e_detected_eyes       VGG  0.017090     85.945651   

                 total_time  
0 0 days 00:16:23.117301226  
1 0 days 00:20:05.628188372  
2 0 days 00:19:24.024507523  
3 0 days 00:08:47.432441950  
4 0 days 00:09:55.128549576  
5 0 days 00:15:27.017767429  
6 0 days 00:08:44.347270250  
7 0 days 00:09:53.161640406  
8 0 days 00:15:16.971753836  


| No |            dataset |    model | avg_loss | avg_accuracy |                  total_time |
|---:|-------------------:|---------:|---------:|-------------:|----------------------------:|
|  0 |          gi4e_full |   ResNet | 0.000017 |    99.999863 | 00:16:23.117301226 | 
|  1 |          gi4e_full | DenseNet | 0.000099 |    99.996225 | 00:20:05.628188372 | 
|  2 |          gi4e_full |      VGG | 0.030500 |    96.777968 | 00:19:24.024507523 | 
|  3 |      gi4e_raw_eyes |   ResNet | 0.017712 |    99.769400 | 00:08:47.432441950 | 
|  4 |      gi4e_raw_eyes | DenseNet | 0.007763 |    99.778149 | 00:09:55.128549576 | 
|  5 |      gi4e_raw_eyes |      VGG | 0.054540 |    81.848047 | 00:15:27.017767429 | 
|  6 | gi4e_detected_eyes |   ResNet | 0.006875 |    99.928385 | 00:08:44.347270250 | 
|  7 | gi4e_detected_eyes | DenseNet | 0.000107 |    99.968702 | 00:09:53.161640406 | 
|  8 | gi4e_detected_eyes |      VGG | 0.017090 |    85.945651 | 00:15:16.971753836 | 



# Embedded Classification

- Load the dataset

In [ ]:
datasets = {
    'gi4e_full': ds.Gi4eDataset(
        './datasets/gi4e',
        transform=transforms.Compose([transforms.ToPILImage(), transforms.Resize((224, 224)), transforms.ToTensor()]),
        is_classification=True),
    'gi4e_raw_eyes': ds.ImageDataset(
        './datasets/gi4e_raw_eyes',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
    'gi4e_detected_eyes': ds.ImageDataset(
        './datasets/gi4e_eyes/20250521_200316',
        transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]),
        file_extension='png'),
}

- Define the model

In [ ]:
embedded_models = [md.FeatureExtractor(model) for model in models]
classifier_models = [md.Classifier(model, 103) for model in embedded_models]


- Load the trained weight to the model

In [ ]:
# ignore

- Transform the dataset to the embedded data

In [ ]:
classifier_df = pd.DataFrame(columns=['key', 'dataset', 'model'])

# Loop through datasets and models to create the classifier_df
print('Starting to get features for datasets...')
for name, dataset in datasets.items():
	for model in embedded_models:
		print(f'Getting features for {name} dataset with {model._get_name()}')

		# get the classifier model from the model
		classifier_model = next((m for m in classifier_models if m.backbone._get_name() == model._get_name()), None)
		if classifier_model is None:
			print(f'Classifier model not found for {model._get_name()}')
			continue

		model_dataset = ds.EmbeddedDataset(dataset, model)

		classifier_df = pd.concat([classifier_df, pd.DataFrame({
				'key': [f'{name}_{model._get_name()}'],
				'model': [classifier_model],
				'dataset': [model_dataset]
		})], ignore_index=True)

		print(f'Finished getting features for {name} dataset with {model._get_name()}')


Starting to get features for datasets...
Getting features for gi4e_full dataset with FeatureExtractor(ResNet)


<All keys matched successfully>


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

Finished training all models

| No|        dataset|                                   model|  avg_loss|  avg_accuracy|  total_time|
|---|---------------|----------------------------------------|----------|--------------|------------|
| 1 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.005883|     99.818135|    4.223327|
| 2 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.012252|     99.677733|   16.936388|
| 3 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.002937|     98.017509|  155.980501|
| 4 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.212613|     95.163104|   16.447177|
| 5 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.221752|     94.722049|    5.249017|
| 6 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.033393|     96.928095|  145.743802|
| 7 |EmbeddedDataset|    Classifier(FeatureExtractor(ResNet))|  0.057876|     97.631673|   18.927852|
| 8 |EmbeddedDataset|  Classifier(FeatureExtractor(DenseNet))|  0.024092|     98.713500|    6.274447|
| 9 |EmbeddedDataset|       Classifier(FeatureExtractor(VGG))|  0.017790|     96.159436|  137.484042|

- Train all defined model on each registered datasets

In [ ]:
result_df = pd.DataFrame(columns=['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time'])

for index, row in classifier_df.iterrows():
	dataset = row['dataset']
	model = row['model']
	key = row['key']

	print(f'Running {key} dataset with {model._get_name()}')
	# do the train
	start_time = time.time()
	scored, loss = train_process(dataset, model)
	end_time = time.time()
	total_time = end_time - start_time
	print(f'Finished {key} dataset with {model._get_name()}')
	print('----------------------')

	# save the result
	result_df = pd.concat([result_df, pd.DataFrame({
			'model': [model._get_name()],
			'dataset': [dataset.__class__.__name__],
			'avg_loss': [loss],
			'avg_accuracy': [scored],
			'total_time': [total_time]
	})], ignore_index=True)
print('Finished training all models')


- Print the result

In [ ]:
# swap the first two columns
result_df = result_df[['dataset', 'model', 'avg_loss', 'avg_accuracy', 'total_time']]
# scale the avg_accuracy to 0-100
result_df['avg_accuracy'] = 100 * (1 - result_df['avg_accuracy'])
# display the total time in the format HH:MM:SS
result_df['total_time'] = pd.to_timedelta(result_df['total_time'], unit='s')

# save the result to csv
result_df.to_csv(f'results_{time_str}.csv', index=False)
# print the result
print(result_df)


In [ ]:
models = [
	torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.DEFAULT),
  torchvision.models.densenet121(weights=torchvision.models.DenseNet121_Weights.DEFAULT),
  torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.DEFAULT)
]

for model in models:
  # explore the model
	print(f'Model: {model.__class__.__name__}')
	
	# explain the model
	print(f'Number of layers: {len(list(model.children()))}')
	print(model)